# LCMS CONUS Image Download Setup
This notebook sets up the environment and parameters for downloading and visualizing LCMS CONUS images, including land cover, land use, and change detection outputs.

In [2]:
# Import required libraries and initialize Earth Engine
import ee,requests,os,sys,shutil,threading
from IPython.display import Image
ee.Initialize(project='lcms-292214',)

# Import geeViz libraries
import geeViz.getImagesLib as gil
import geeViz.changeDetectionLib as cdl
Map = gil.Map

# Set output folder and version
output_folder = '../assets/images'
version = '2024-10'
# Define output file paths
# conus_loss = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss.png')
# conus_loss_gain = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss_Gain.png')
# conus_change = os.path.join(output_folder,f'CONUS_v{version}_Change.png')
# conus_lc = os.path.join(output_folder,f'CONUS_v{version}_Land_Cover.png')
# conus_lc3 = os.path.join(output_folder,f'CONUS_v{version}_Land_Cover3.png')
# conus_lu = os.path.join(output_folder,f'CONUS_v{version}_Land_Use.png')

## Visualization Parameters
Define the color palettes, class names, and values for the different LCMS outputs (Change, Land Cover, Land Use).

In [4]:
# Visualization parameters for LCMS outputs
viz = {
    # Change class names, palettes, and values
    "Change_class_names": [
        "Wind",
        "Hurricane",
        "Snow or Ice Transition",
        "Desiccation",
        "Inundation",
        "Prescribed Fire",
        "Wildfire",
        "Mechanical Land Transformation",
        "Tree Removal",
        "Defoliation",
        "Southern Pine Beetle",
        "Insect, Disease, or Drought Stress",
        "Other Loss",
        "Vegetation Successional Growth",
        "Stable",
        "Non-Processing Area Mask"
    ],
    "Change_class_palette": [
        "FF09F3",
        "541AFF",
        "E4F5FD",
        "CC982E",
        "0ADAFF",
        "A10018",
        "D54309",
        "FAFA4B",
        "AFDE1C",
        "FFC80D",
        "A64C28",
        "F39268",
        "C291D5",
        "00A398",
        "3D4551",
        "1B1716"
    ],
    "Change_class_values": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16
    ],
    # Land Cover 3-class names, palettes, and values
    "Land_Cover3_class_names": [
        "Tree",
        "Shrub",
        "Grass/Forb/Herb",
        "Barren or Impervious",
        "Snow or Ice",
        "Water",
        "Non-Processing Area Mask"
    ],
    "Land_Cover3_class_palette": [
        "004E2B",
        "F89A1C",
        "E5E98A",
        "893F54",
        "E4F5FD",
        "00B6F0",
        "1B1716"
    ],
    "Land_Cover3_class_values": [
        1,
        2,
        3,
        4,
        5,
        6,
        7
    ],
    # Land Cover class names, palettes, and values
    "Land_Cover_class_names": [
        "Tree",
        "Tall Shrub & Tree Mix (AK Only)",
        "Shrub & Tree Mix",
        "Grass/Forb/Herb & Tree Mix",
        "Barren & Tree Mix",
        "Tall Shrub (AK Only)",
        "Shrub",
        "Grass/Forb/Herb & Shrub Mix",
        "Barren & Shrub Mix",
        "Grass/Forb/Herb",
        "Barren & Grass/Forb/Herb Mix",
        "Barren or Impervious",
        "Snow or Ice",
        "Water",
        "Non-Processing Area Mask"
    ],
    "Land_Cover_class_palette": [
        "004E2B",
        "009344",
        "61BB46",
        "ACBB67",
        "8B8560",
        "CAFD4B",
        "F89A1C",
        "8FA55F",
        "BEBB8E",
        "E5E98A",
        "DDB925",
        "893F54",
        "E4F5FD",
        "00B6F0",
        "1B1716"
    ],
    "Land_Cover_class_values": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15
    ],
    # Land Use class names, palettes, and values
    "Land_Use_class_names": [
        "Agriculture",
        "Developed",
        "Forest",
        "Other",
        "Rangeland or Pasture",
        "Non-Processing Area Mask"
    ],
    "Land_Use_class_palette": [
        "FBFF97",
        "E6558B",
        "004E2B",
        "9DBAC5",
        "A6976A",
        "1B1716"
    ],
    "Land_Use_class_values": [
        1,
        2,
        3,
        4,
        5,
        6
    ]
}

# Make a copy and remove Change viz for 2023.9 LCMS data (PRUSVI and HI)
viz_hybrid = viz.copy()
del viz_hybrid['Change_class_values']
del viz_hybrid['Change_class_names']
del viz_hybrid['Change_class_palette']
print(viz_hybrid)

{'Land_Cover3_class_names': ['Tree', 'Shrub', 'Grass/Forb/Herb', 'Barren or Impervious', 'Snow or Ice', 'Water', 'Non-Processing Area Mask'], 'Land_Cover3_class_palette': ['004E2B', 'F89A1C', 'E5E98A', '893F54', 'E4F5FD', '00B6F0', '1B1716'], 'Land_Cover3_class_values': [1, 2, 3, 4, 5, 6, 7], 'Land_Cover_class_names': ['Tree', 'Tall Shrub & Tree Mix (AK Only)', 'Shrub & Tree Mix', 'Grass/Forb/Herb & Tree Mix', 'Barren & Tree Mix', 'Tall Shrub (AK Only)', 'Shrub', 'Grass/Forb/Herb & Shrub Mix', 'Barren & Shrub Mix', 'Grass/Forb/Herb', 'Barren & Grass/Forb/Herb Mix', 'Barren or Impervious', 'Snow or Ice', 'Water', 'Non-Processing Area Mask'], 'Land_Cover_class_palette': ['004E2B', '009344', '61BB46', 'ACBB67', '8B8560', 'CAFD4B', 'F89A1C', '8FA55F', 'BEBB8E', 'E5E98A', 'DDB925', '893F54', 'E4F5FD', '00B6F0', '1B1716'], 'Land_Cover_class_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'Land_Use_class_names': ['Agriculture', 'Developed', 'Forest', 'Other', 'Rangeland or Pastu

## Prepare and Visualize CONUS Change Image
Load the composite and LCMS image collections, apply masks, visualize, and start downloading the CONUS change image.

In [5]:
# Get CONUS Loss and Loss+Gain images


dem = ee.Image("USGS/3DEP/10m").resample("bicubic")
hillshade = ee.Terrain.hillshade(dem)



# Get composite image for CONUS
comp_collections = {"CONUS":"projects/lcms-tcc-shared/assets/CONUS/Composites/Composite-Collection-yesL7",
      "AK":"projects/lcms-tcc-shared/assets/OCONUS/R10/AK/Composites/Composite-Collection",
      "PRUSVI":"projects/lcms-tcc-shared/assets/OCONUS/R8/PR_USVI/Composites/Composite-Collection",
      "HAWAII":"projects/lcms-tcc-shared/assets/OCONUS/Hawaii/Composites/Composite-Collection1988-2023",}
lcms_collections = {"CONUS":"projects/lcms-292214/assets/Final_Outputs/2024-10/LCMS",
                    "AK":"projects/lcms-292214/assets/Final_Outputs/2024-10/LCMS",
                    "PRUSVI":"USFS/GTAC/LCMS/v2023-9",
                    "HAWAII":"USFS/GTAC/LCMS/v2023-9",}
# Number of pixels (at view pyramid level) to buffer change by
change_buffer={'CONUS':1,
                'AK':0,
                'PRUSVI':0,
                'HAWAII':1,}
# Number of pixels (at view pyramid level) to include as change value. 
# Higher number will show more change
change_reduce_res_max_pixels={'CONUS':8,
                'AK':5,
                'PRUSVI':4,
                'HAWAII':4,}

comp_viz = {
   'CONUS': {'min': 100, 'max': [5000.0, 5000.0, 4400.0], 'bands': 'red,green,blue',},
   'AK': {'min': 50, 'max': [3500.0, 3500.0, 3800.0], 'bands': 'red,green,blue',},
    'PRUSVI': {'min': 50, 'max': [3500.0, 3500.0, 3800.0], 'bands': 'red,green,blue',},
    'HAWAII': {'min': 50, 'max': [3000.0, 3000.0, 3500.0], 'bands': 'red,green,blue',},
}



#####################################################################################
def downloadImage(url, filename):
    """Downloads an image from a URL and saves it to a file.

    Args:
        url (str): Image URL.
        filename (str): Output file path.
    """
    img_data = requests.get(url).content
    with open(filename, 'wb') as handler:
        handler.write(img_data)


def downloadChangePng(sa_name,size=1000):
  output_name_change = os.path.join(output_folder,f'{sa_name}_v{version}_Change.png')

  comp = ee.ImageCollection(comp_collections[sa_name])\
  .filter(ee.Filter.calendarRange(2021,2024,'year')).mean()

  Map.clearMap()
  Map.addLayer(comp,comp_viz[sa_name],'Comp Raw')

  # Load LCMS image collection and set visualization parameters
  lcms = ee.ImageCollection(lcms_collections[sa_name]).filter(f"study_area=='{sa_name}'")
  sa = lcms.first().geometry()
  projection = lcms.first().projection().getInfo()
  crs = projection['wkt']
  transform = projection['transform']

  # Set new viz props
  if sa_name == 'CONUS' or sa_name == 'AK':
      lcms = lcms.map(lambda img:img.set(viz))



  # Create mask for valid data
  msk = lcms.first().select([0]).mask()
  msk = msk.focal_min(1.5,'circle','pixels')

  # Mask composite image
  comp = comp.updateMask(msk)
  comp = comp.visualize(**comp_viz[sa_name])

  Map.addLayer(comp,{},'Comp Raw Contracted')
  Map.addLayer(msk,{},'MSK')

  # Get properties and visualize Land Cover and Land Use
  props = lcms.first().toDictionary()
 
  # Get minimum change image and mask

  if sa_name == 'CONUS' or sa_name == 'AK':
    all_change = lcms.select(['Change']).min()
    all_change = all_change.setDefaultProjection(crs,transform)

    change_msk = all_change.lte(14)
  else:
    all_change = lcms.select(['Change'])
    all_change = all_change.map(lambda img:img.updateMask(img.gt(1).And(img.lt(5)))).min()
    all_change = all_change.setDefaultProjection(crs,transform)
    change_msk = all_change.gt(1).And(all_change.lt(5))
  all_change = all_change.updateMask(change_msk).reduceResolution(ee.Reducer.min(),True,change_reduce_res_max_pixels[sa_name]).updateMask(change_msk.focal_max(change_buffer[sa_name]).And(msk))
  all_change = all_change.set(props).visualize()

  all_change_msk = all_change.mask()
  all_change = all_change.unmask(comp)
  Map.addLayer(all_change,{},'All Change')
  Map.turnOnInspector()

  # Set parameters for thumbnail download
  params = {
    'region': sa.bounds(500,crs),
    'dimensions': size,
    'crs': crs,
    'transform':transform
  }

  # Download all change image as thumbnail
  all_change_url = all_change.getThumbURL(params)
  t = threading.Thread(target = downloadImage,args=(all_change_url,output_name_change))
  t.start()
  print(all_change_url)
  display(Image(url=all_change_url))

###################################################
for sa_name in comp_collections.keys():
  downloadChangePng(sa_name)

Adding layer: Comp Raw
Adding layer: Comp Raw Contracted
Adding layer: MSK
Adding layer: All Change
https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/fdd802c68e29e21f8536aacdb37da72d-8a20d8dabdce0d9920b0c048ad414815:getPixels


Adding layer: Comp Raw
Adding layer: Comp Raw Contracted
Adding layer: MSK
Adding layer: All Change
https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/2a4e709af2186580fc50475947b42b89-494a56d17acb6834b724452a9c6e25a0:getPixels


Adding layer: Comp Raw
Adding layer: Comp Raw Contracted
Adding layer: MSK
Adding layer: All Change
https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/000b2d5be119a0ba168e57869a97a850-c1aa39af4d53bf056c3f2e680a33cb4b:getPixels


Adding layer: Comp Raw
Adding layer: Comp Raw Contracted
Adding layer: MSK
Adding layer: All Change
https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/a8da23b253b1db1026141c741224ae50-06e0b1894df2abb48168e99228218fef:getPixels


## Check Active Threads
Print the number of active threads (for monitoring downloads).

In [8]:
# Print the number of active threads (for monitoring downloads)
print(threading.active_count())

6


In [8]:
# Download Land Cover and Land Use images

def downloadLULCPng(sa_name,size=1000):
    lcms = ee.ImageCollection(lcms_collections[sa_name]).filter(f"study_area=='{sa_name}'")
    sa = lcms.first().geometry()
    projection = lcms.first().projection().getInfo()
    crs = projection['wkt']
    transform = projection['transform']

    if sa_name == 'CONUS' or sa_name == 'AK':
        lcms = lcms.map(lambda img:img.set(viz))
    else:
        lcms = lcms.map(lambda img:img.set(viz_hybrid))
        lcms = lcms.map(lambda img:
        img.addBands(
            img
              .select(["Land_Use"])
              .remap([1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 4, 5, 6])
              .rename(["Land_Use"]),
            None,
            True
          )
      )
    params = {
      'region': sa.bounds(500,crs),
      'dimensions': 1000,
      'crs': crs,
      'transform':transform
    }
    for bn in ['Land_Cover','Land_Use']:
        # Get latest Land Cover image
        t = lcms.select([bn]).sort('system:time_start',False).first().visualize()

        # Get thumbnail URL
        url = t.getThumbURL(params)

        # Download image in separate thread
        output_name = os.path.join(output_folder,f'{sa_name}_v{version}_{bn}.png')
        t = threading.Thread(target = downloadImage,args=(url,output_name))
        t.start()
        print(url)
        display(Image(url=url))

for sa_name in comp_collections.keys():
    downloadLULCPng(sa_name,size=1000)

https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/c235ce6b30162dbff4cb9e4cb966d842-7100839874e54d53551464141c17a050:getPixels


https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/a10da02b953540231449bcfd7b586cb1-a079a804a5cb446e216752809adf9b81:getPixels


https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/64ba1d97b091f70591b42f597d62d004-c58c4a327f367de58625f30f208922f1:getPixels


https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/bced5b0273a82a0b332f2ad3ca254e14-180921fb9ae01fb4ee9a0745633eb216:getPixels


https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/b6511455405b474991f6ae04d6582086-a2408b1d6d8ed92b3cd48a425c243e15:getPixels


https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/e85811d03a4be9b83dadc00b686dc5cc-10a1fc0d413a51ac94149297aba5885e:getPixels


https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/afe58838267c932728922a4f1a0acd83-f0ee49d97561d669ddbdf41bd99911cb:getPixels


https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/1a2a1e0a16053b4c871889a8d21b9d4f-94dc642cff9f5e03baa06ac8ffc6e7fb:getPixels


## Move Example GIFs to Assets Folder
Copy example GIFs from a source directory to the assets folder for use in outreach materials.

In [ ]:
# Move gifs from gif folder to assets folder
from_dir = r'Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs'

which_ones = ['Atlanta-Land_Use_wYears.gif','Great_Salt_Lake-Land_Cover_wYears.gif','Northern_Colorado-Change_wYears.gif','Columbia_Glacier-Land_Cover_wYears.gif','Lake_Mead-Land_Cover_wYears.gif','Oregon_Wildfire-Change_wYears.gif','Las_Vegas-Land_Use_wYears.gif','Fountain_Wildfire-Change_wYears.gif','Denver-Land_Use_wYears.gif']

for w in which_ones:
    # Copy each gif to the output folder
    print('Copying:',os.path.basename(w))
    input = os.path.join(from_dir,os.path.basename(w))
    output = os.path.join(output_folder,os.path.basename(w))
    print(input,output)
    shutil.copy2(input,output)

Copying: Atlanta-Land_Use_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Atlanta-Land_Use_wYears.gif ../assets/images\Atlanta-Land_Use_wYears.gif
Copying: Great_Salt_Lake-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Great_Salt_Lake-Land_Cover_wYears.gif ../assets/images\Great_Salt_Lake-Land_Cover_wYears.gif
Copying: Northern_Colorado-Change_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Northern_Colorado-Change_wYears.gif ../assets/images\Northern_Colorado-Change_wYears.gif
Copying: Columbia_Glacier-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Columbia_Glacier-Land_Cover_wYears.gif ../assets/images\Columbia_Glacier-Land_Cover_wYears.gif
Copying: Lake_Mead-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\